## Data

In [1]:
from data import get_data_paths, Dataloder
from sklearn.model_selection import train_test_split

In [2]:
%%time
data_paths = get_data_paths()

# split data
train_paths, test_paths = train_test_split(data_paths, test_size=0.2, random_state=42)
test_paths, val_paths = train_test_split(test_paths, test_size=0.5, random_state=42)

print(f"Train: {len(train_paths)}, Val: {len(val_paths)}, Test: {len(test_paths)}")

# Dataloaders
train_dataloader = Dataloder(train_paths, batch_size=16, shuffle=True)
val_dataloader = Dataloder(val_paths, batch_size=16, shuffle=True)
test_dataloader = Dataloder(test_paths, batch_size=1, shuffle=False)

Train: 13245, Val: 1656, Test: 1656
CPU times: user 3.27 s, sys: 8.78 ms, total: 3.28 s
Wall time: 3.28 s


## Model

In [3]:
from ddrnet_23_slim import get_ddrnet_23_slim_model
from shared import validate_model, get_callbacks

Segmentation Models: using `keras` framework.


In [4]:
import tensorflow as tf

gpu = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [5]:
# get model
model = get_ddrnet_23_slim_model()
# get callbacks
callbacks = get_callbacks(save_model_path="../data/models/ddrnet_23_slim.hs",
                          log_dir="../data/tensorboard_logs/ddrnet_23_slim/")

2022-04-03 16:14:02.540929: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-03 16:14:02.859751: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10029 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:01:00.0, compute capability: 8.6


In [6]:
%%time
# fit model
history = model.fit(train_dataloader,
                    steps_per_epoch=len(train_dataloader),
                    epochs=30,
                    validation_data=val_dataloader,
                    validation_steps=len(val_dataloader),
                    callbacks=callbacks)

Epoch 1/30


2022-04-03 16:14:06.920752: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200
2022-04-03 16:14:07.555352: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


827/827 [==============================] - 132s 153ms/step - loss: 0.5153 - iou_score: 0.3923 - f1-score: 0.5192 - val_loss: 0.3531 - val_iou_score: 0.5303 - val_f1-score: 0.6742 - lr: 0.0010
Epoch 2/30
827/827 [==============================] - 128s 155ms/step - loss: 0.2926 - iou_score: 0.5981 - f1-score: 0.7300 - val_loss: 0.2696 - val_iou_score: 0.6218 - val_f1-score: 0.7509 - lr: 0.0010
Epoch 3/30
827/827 [==============================] - 126s 153ms/step - loss: 0.2434 - iou_score: 0.6552 - f1-score: 0.7760 - val_loss: 0.2295 - val_iou_score: 0.6728 - val_f1-score: 0.7909 - lr: 0.0010
Epoch 4/30
827/827 [==============================] - 126s 153ms/step - loss: 0.2102 - iou_score: 0.6993 - f1-score: 0.8099 - val_loss: 0.2063 - val_iou_score: 0.7041 - val_f1-score: 0.8153 - lr: 0.0010
Epoch 5/30
827/827 [==============================] - 127s 153ms/step - loss: 0.1923 - iou_score: 0.7215 - f1-score: 0.8259 - val_loss: 0.1979 - val_iou_score: 0.7149 - val_f1-score: 0.8226 - lr: 0.0

In [7]:
%%time
test_metrics = validate_model(model, test_dataloader)
test_metrics

1656/1656 [==============================] - 14s 8ms/step - loss: 0.2336 - iou_score: 0.7692 - f1-score: 0.8318
CPU times: user 25.2 s, sys: 624 ms, total: 25.8 s
Wall time: 14.2 s


{'inference time': 0.008600651641974703,
 'loss': 0.2335771918296814,
 'IoU': 0.7691971063613892,
 'F1': 0.8317785859107971}